In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

print("Current working directory:", os.getcwd())
load_dotenv()

sarvam_api_key = os.getenv("SARVAM_API_KEY")
if sarvam_api_key:
    print('SARVAM API KEY FOUND')
else:
    print("No SARVAM API key found")

grok_api_key = os.getenv("GROK_API_KEY")
if grok_api_key:
    print('GROK API KEY FOUND')
else:
    print("No GROK API key found")

serp_dev_api_key = os.getenv("SERP_DEV_API_KEY")
if serp_dev_api_key:
    print('SERP DEV API KEY FOUND')
else:
    print("No SERP API key found")

pd.set_option('display.max_colwidth', 1000)

#model_multi_query = 'llama3-8b-8192'
#model_summarizer = 'llama3-8b-8192'
#model_judge = 'llama3-8b-8192'

Current working directory: c:\Users\rahul\OneDrive\7_Learning\IISC\Courses\3.1_Deep_Learning\Course Material\Project\wip
SARVAM API KEY FOUND
GROK API KEY FOUND
SERP DEV API KEY FOUND


In [16]:
def model_evaluate(train_file_name='fake_news_claims_factchecked.csv'):
    """
       Description: This function evaluates the model performance and returns the dataframe
    """
    models = ['llama3-8b-8192','qwen/qwen3-32b','mistral-saba-24b','deepseek-r1-distill-llama-70b',
              'meta-llama/llama-4-scout-17b-16e-instruct',  'meta-llama/llama-4-maverick-17b-128e-instruct',  'qwen-qwq-32b']
    
    df_model_res =pd.DataFrame(columns=['model_multi_query', 'model_summarizer', 'model_judge', 'abstention_rate', 'accuracy', 'overall_accuracy', 'coverage', 'abstention_rate', 'f1_real', 'f1_fake'])    
    df_res = pd.DataFrame(columns=['model_judge','claim', 'verdict_orig','verdict_pred','explanation'])
    
    for judge_model in models:
        model_multi_query = models[0]  # Keep constant
        model_summarizer = models[1]   # Keep constant  
        model_judge = judge_model      # Test each one
        print(f"Testing with judge: {judge_model}")
    
        
        df_test = pd.read_csv(train_file_name)
        df_test = df_test.head(1)  # Limit to 50 rows for testing
        
        for index, row in df_test.iterrows():
            lis1 = []
            claim = row.claim
            print(f"Processing row {index + 1}/{len(df_test)} Using judge model: {judge_model}")
    
            claim_final, verdict_class, verdict_full = verify_news(claim,model_multi_query,model_summarizer,model_judge)
            lis1 = [model_judge , claim, row.label, verdict_class, verdict_full]
        
            df_res.loc[len(df_res)] = lis1
    
        df_res.verdict_orig = df_res.verdict_orig.str.upper()
        y_true_labels = df_res['verdict_orig'].tolist()
        y_pred_labels = df_res['verdict_pred'].tolist()
    
        total = len(df_res)
    
        # Filter each category
        TP = len(df_res[(df_res.verdict_orig == "REAL") & (df_res.verdict_pred == "REAL")])
        TN = len(df_res[(df_res.verdict_orig == "FAKE") & (df_res.verdict_pred == "FAKE")])
        FP = len(df_res[(df_res.verdict_orig == "FAKE") & (df_res.verdict_pred == "REAL")])
        FN = len(df_res[(df_res.verdict_orig == "REAL") & (df_res.verdict_pred == "FAKE")])
        U1 = len(df_res[(df_res.verdict_orig == "REAL") & (df_res.verdict_pred == "UNSURE")])
        U2 = len(df_res[(df_res.verdict_orig == "FAKE") & (df_res.verdict_pred == "UNSURE")])
    
        # Base metrics
        confident_preds = TP + TN + FP + FN
        unsure_preds = U1 + U2
    
        accuracy = (TP + TN) / confident_preds if confident_preds else 0
        overall_accuracy = (TP + TN) / total if total else 0
        coverage = confident_preds / total
        abstention_rate = unsure_preds / total
    
        # F1 for REAL class
        precision_real = TP / (TP + FP) if (TP + FP) else 0
        recall_real = TP / (TP + FN) if (TP + FN) else 0
        f1_real = 2 * precision_real * recall_real / (precision_real + recall_real) if (precision_real + recall_real) else 0
    
        # F1 for FAKE class
        TP_fake = TN
        FP_fake = FN
        FN_fake = FP + U2
    
        precision_fake = TP_fake / (TP_fake + FP_fake) if (TP_fake + FP_fake) else 0
        recall_fake = TP_fake / (TP_fake + FN_fake) if (TP_fake + FN_fake) else 0
        f1_fake = 2 * precision_fake * recall_fake / (precision_fake + recall_fake) if (precision_fake + recall_fake) else 0
    
        list_res = [model_multi_query, model_summarizer, judge_model,abstention_rate,accuracy,overall_accuracy,coverage,abstention_rate,f1_real,f1_fake]
        df_model_res.loc[len(df_model_res)] = list_res

    return df_model_res

Evaluating with Prompt Strategy 1  
    
    Multi Query RAG  
    Summarization  
    Judge Prompt  

In [17]:
import requests
import mimetypes
import os
import json

from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

grok_api_key = os.getenv('GROK_API_KEY')
serp_dev_api_key = os.getenv('SERP_DEV_API_KEY')
sarvam_api_key = os.getenv('SARVAM_API_KEY')

empty_search = None

class SerperRetrieverWrapper_1:
    #Class to use Serper as retriver agent for the RAG framework
    def __init__(self, api_key: str, num_results: int = 15):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        """
        try:
            _SERPER_SEARCH_URL = "https://google.serper.dev/news"
            q1 = '(site:news18.com OR site:ptinews.com OR site:politifact.com) ' + query
            #print('Serper query: ', q1)
    
            headers = {
                "X-API-KEY": self.api_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": q1,
                "num": self.num_results,
            }
            
            resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload)
            #print('Serper response: ', resp)
            #print('Serper response code: ', resp.json())
            
            if resp.status_code != 200:
                raise Exception(f"Serper API Error: {resp.text}")
            results = resp.json()
            #print('Serper results: ', results.type)

            #return results
            
            documents = []
            for i, item in enumerate(results.get("news", []), 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')
                documents.append(Document(
                    page_content=f"{title}\n{snippet}",
                    metadata={"source": link}
                ))
            
            #if not documents:
            #    empty_search = Exception("No result found")
            #    raise empty_search
            
            #print(f"Retrieved {len(documents)} documents from Serper.")
            return documents
        except Exception as e:
            if e is empty_search:
                print('serper main exception1')
                raise empty_search
            else:
                print('serper main exception2')
                raise e
        
def verify_news(user_claim, i_model_multi_query, i_model_summarizer, i_model_judge, input_lang = 'auto'):
    """
       Description: This function is used to verify the claim provided by the user and output as REAL or FAKE or UNSURE based on the context with a short explanation
       INPUT: user_claim --> The news user wish to verify
       OUTPUT: FAKE/REAL/UNSURE with explanation
    """
    try:
       model_multi_query = i_model_multi_query
       model_summarizer = i_model_summarizer
       model_judge = i_model_judge

       #print('here1: ', user_claim)
       claim1 = user_claim.replace("'","")
       claim1 = user_claim.replace("\n"," ")
       #print('here2: ', claim1)
        
       serper_retriever = SerperRetrieverWrapper_1(api_key=serp_dev_api_key)
       context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)
       
       """
       Implemeting RAG framework with Multi Query Translation
       Step 1 -- Take input from user for the news to verify
       Step 2 -- Generate 3 variants of the news for better seach results
       Step 3 -- Summarise the results from all the previous steps to be passed to main prompt
       Step 4 -- Deliver the final verdict with a short explanation
       """
       
       #Multi Query Generation
       
       multi_query_template = """
            You are an AI language assistant.

             Your task is to generate three alternative phrasings of the given user question to help retrieve relevant documents from a vector database. These alternatives should reflect different ways the original question might be asked, using varied vocabulary or structure, while preserving the original intent.             
             This helps improve the chances of matching relevant content in a distance-based similarity search.
             
             Original question: {question}
             
             Output:
             Provide exactly three reworded versions of the question, each on a new line.
       """
       perspectives_prompt = ChatPromptTemplate.from_template(multi_query_template)
       
       llm_multi_query = ChatGroq(api_key = grok_api_key, model_name = model_multi_query)
       
       generate_queries = (
           perspectives_prompt 
           | llm_multi_query
           | StrOutputParser() 
           | (lambda x: x.split("\n"))
       )
       
       #Summarization using multi query
       
       summarizer_template = '''
          You are an assistant summarizing factual evidence from multiple documents.
       
          Based on the following documents, extract the key facts relevant to the claim.
          
          Claim: {question}
          
          Documents:
          {context}
          
          Return a short neutral summary of the key facts only.
       '''
       summarizer_prompt = PromptTemplate.from_template(summarizer_template)
       
       llm_summarizer = ChatGroq(api_key = grok_api_key, model_name = model_summarizer)
       summarizer_chain = (
           {
               "context": context_retriever,
               "question": generate_queries
           }
           | summarizer_prompt
           | llm_summarizer
           | StrOutputParser()
       )
       
       #Final Judgement 
       
       fact_checker_template = '''
          You are a fact-checking assistant. Give a direct answer without showing your thinking process.
          
          Claim: {question}
          
          Evidence:
          {evidence}
          
          Instructions:
          
          If evidence is provided, use only that evidence to determine whether the claim is REAL, FAKE, or UNSURE.          
          If no evidence is provided, conduct a brief web search to gather supporting or contradicting information. Prioritize information from reliable, reputable sources such as news websites, fact-checking organizations, or official statements.
          
          If no sufficient or conclusive evidence is found from these sources, respond with UNSURE.          
          Do not speculate or rely on untrusted sources.

          Respond in this format:
          
          Classification: REAL / FAKE / UNSURE  
          Explanation: <Your reasoning, clearly supported by the evidence or source content> 
       '''
       
       fact_checker_prompt = PromptTemplate.from_template(fact_checker_template)
       
       llm_fact_checker = ChatGroq(api_key = grok_api_key, model_name = model_judge)
       fact_checker_chain = (
           {
               "question": RunnablePassthrough(),
               "evidence": summarizer_chain 
           }
           | fact_checker_prompt
           | llm_fact_checker
           | StrOutputParser()
       )
   
       claim = claim1
       
       #Calling SARVAM API to translate Indic languages to English
       client = SarvamAI(api_subscription_key = sarvam_api_key)
       
       try:
           translation = client.text.translate(
           input=claim,
           source_language_code="auto",
           target_language_code="en-IN"
           )
       except Exception as e:
           print(f"Error during translation: {e}")
           error_msg = 'It appears you have provided input in an alien language. Please try again with some other language'
           return error_msg,error_msg,error_msg
       
       claim_final = translation.translated_text if translation else claim
       claim_orig_lang = translation.source_language_code
       #print(f"Translated claim: {claim_final}")
       
       verdict_orig = fact_checker_chain.invoke(claim_final)
       verdict_class = verdict_orig.split('\n')[0]
       verdict_explan = verdict_orig.split('\n')[-1]

       # Strip out any <think> or <tool> or similar tags
       cleaned_output = re.sub(r"<[^>]+>", "", verdict_orig).strip()
       
       verdict_class = re.search(r'Classification:\s*(REAL|FAKE|UNSURE)', cleaned_output)
       verdict_explan = re.search(r'Explanation:\s*(.*)', cleaned_output, re.DOTALL)
       
       verdict_class = verdict_class.group(1) if verdict_class else "UNSURE"
       verdict_explan = verdict_explan.group(1).strip() if verdict_explan else "No clear explanation found."

       #print('here1: ',verdict_class)
       #print('here2: ',verdict_explan)
       
       if input_lang == 'auto':
           trans_lang = claim_orig_lang
       else:
           trans_lang = input_lang
   
       if claim_orig_lang != 'en-IN':
           try:
               translation_class = client.text.translate(
               input=verdict_class,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
               
           try:
               translation_explan = client.text.translate(
               input=verdict_explan,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
           
           verdict_trans_class = translation_class.translated_text
           verdict_trans_explan = translation_explan.translated_text
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           
           verdict_orig = verdict_class + '\n\n' + verdict_explan
       else:
           verdict_orig = verdict_class + '\n\n' + verdict_explan
           verdict_trans_class = verdict_class
           verdict_trans_explan = verdict_explan
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           #print('verdict_orig: ', verdict_orig) 
           
           try:
               match = re.search(r':(.*)', verdict_class)
               if match:
                    verdict1 = match.group(1).strip()
                    #print(verdict1)
           except Exception as e:
               print('here3')
               verdict1 = verdict_class
       
       #return(claim_final, verdict_orig, verdict_trans)
       return(claim_final, verdict_class, verdict_orig)
    except Exception as e:
        if str(e) == 'No result found':
            print('Error in main proc1. Error is ', e)
            error_msg = 'The search for this claim came back empty. Please rephrase the claim or try with a new one'
            return('UNSURE' ,error_msg,error_msg)
        else:
            print('Error in main proc2. Error is ', e)
            error_msg = 'Something went wrong. Please try after some time'
            return(error_msg,error_msg,error_msg)
    
def transcribe_audio(audio):
    """
       Description: This function trascibes audio using SarvamAI STT model 
    """
    try:
        client = SarvamAI(api_subscription_key = sarvam_api_key)
        mime_type, _ = mimetypes.guess_type(audio)
        
        with open(audio, "rb") as f:
            response = client.speech_to_text.transcribe(
                file=("audio.mp3", f, mime_type or "audio/mpeg"),
                model="saarika:v2.5",
                language_code="unknown"
            )
        ret_var = response.transcript
        ret_lang = response.language_code
    except Exception as e:
        print(f"Error during translation: {e}")
        ret_var = ''

    return ret_var, ret_lang
    
def verify_news_audio(audio):
    """
       Description: This function verifies the news where input method is Audio
    """
    
    claim, orig_lang = transcribe_audio(audio)
    if claim == '':
        error_msg = 'I could not understand your message. Please try recording again'
        return(error_msg,error_msg,error_msg)
    
    final_claim, verdict, verdict_trans = verify_news(claim, orig_lang)
    return final_claim, verdict, verdict_trans

In [ ]:
df_strategy_1 = model_evaluate()
df_strategy_1

Evaluating with Prompt Strategy 2
    
    Multi Query RAG    
    Judge Prompt  

In [12]:
import requests
import mimetypes
import os
import json

from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

grok_api_key = os.getenv('GROK_API_KEY')
serp_dev_api_key = os.getenv('SERP_DEV_API_KEY')
sarvam_api_key = os.getenv('SARVAM_API_KEY')


empty_search = None

class SerperRetrieverWrapper_2:
    #Class to use Serper as retriver agent for the RAG framework
    def __init__(self, api_key: str, num_results: int = 15):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        """
        try:
            _SERPER_SEARCH_URL = "https://google.serper.dev/news"
            q1 = '(site:news18.com OR site:ptinews.com OR site:politifact.com) ' + query
            #print('Serper query: ', q1)
    
            headers = {
                "X-API-KEY": self.api_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": q1,
                "num": self.num_results,
            }
            
            resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload)
            #print('Serper response: ', resp)
            #print('Serper response code: ', resp.json())
            
            if resp.status_code != 200:
                raise Exception(f"Serper API Error: {resp.text}")
            results = resp.json()
            #print('Serper results: ', results.type)

            #return results
            
            documents = []
            for i, item in enumerate(results.get("news", []), 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')
                documents.append(Document(
                    page_content=f"{title}\n{snippet}",
                    metadata={"source": link}
                ))
            
            #if not documents:
            #    empty_search = Exception("No result found")
            #    raise empty_search
            
            #print(f"Retrieved {len(documents)} documents from Serper.")
            return documents
        except Exception as e:
            if e is empty_search:
                print('serper main exception1')
                raise empty_search
            else:
                print('serper main exception2')
                raise e
        
def verify_news(user_claim, i_model_multi_query, i_model_summarizer, i_model_judge, input_lang = 'auto'):
    """
       Description: This function is used to verify the claim provided by the user and output as REAL or FAKE or UNSURE based on the context with a short explanation
       INPUT: user_claim --> The news user wish to verify
       OUTPUT: FAKE/REAL/UNSURE with explanation
    """
    try:
       model_multi_query = i_model_multi_query
       model_summarizer = i_model_summarizer
       model_judge = i_model_judge

       #print('here1: ', user_claim)
       claim1 = user_claim.replace("'","")
       claim1 = claim1.replace("\n"," ")
       #print('here2: ', claim1)
        
       serper_retriever = SerperRetrieverWrapper_2(api_key=serp_dev_api_key)
       context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)
       
       """
       Implemeting RAG framework with Multi Query Translation
       Step 1 -- Take input from user for the news to verify
       Step 2 -- Generate 3 variants of the news for better seach results
       Step 3 -- Summarise the results from all the previous steps to be passed to main prompt
       Step 4 -- Deliver the final verdict with a short explanation
       """
       
       #Multi Query Generation
       
       multi_query_template = """
            You are an AI language assistant.

             Your task is to generate three alternative phrasings of the given user question to help retrieve relevant documents from a vector database. These alternatives should reflect different ways the original question might be asked, using varied vocabulary or structure, while preserving the original intent.             
             This helps improve the chances of matching relevant content in a distance-based similarity search.
             
             Original question: {question}
             
             Output:
             Provide exactly three reworded versions of the question, each on a new line.
       """
       perspectives_prompt = ChatPromptTemplate.from_template(multi_query_template)
       
       llm_multi_query = ChatGroq(api_key = grok_api_key, model_name = model_multi_query)
       
       generate_queries = (
           perspectives_prompt 
           | llm_multi_query
           | StrOutputParser() 
           | (lambda x: x.split("\n"))
       )
       
       """
       #Summarization using multi query
       
       summarizer_template = '''
          You are an assistant summarizing factual evidence from multiple documents.
       
          Based on the following documents, extract the key facts relevant to the claim.
          
          Claim: {question}
          
          Documents:
          {context}
          
          Return a short neutral summary of the key facts only.
       '''
       summarizer_prompt = PromptTemplate.from_template(summarizer_template)
       
       llm_summarizer = ChatGroq(api_key = grok_api_key, model_name = model_summarizer)
       summarizer_chain = (
           {
               "context": context_retriever,
               "question": generate_queries
           }
           | summarizer_prompt
           | llm_summarizer
           | StrOutputParser()
       )
       """
       
       #Final Judgement 
       
       fact_checker_template = '''
          You are a fact-checking assistant.
          
          Claim: {question}
          
          Evidence:
          {evidence}
          
          Instructions:
          
          If evidence is provided, use only that evidence to determine whether the claim is REAL, FAKE, or UNSURE.          
          
          If no sufficient or conclusive evidence is found from these sources, respond with UNSURE.          
          Do not speculate or rely on untrusted sources.

          Respond in this format:
          
          Classification: REAL / FAKE / UNSURE  
          Explanation: <Your reasoning, clearly supported by the evidence or source content> 
       '''
       
       fact_checker_prompt = PromptTemplate.from_template(fact_checker_template)
       
       llm_fact_checker = ChatGroq(api_key = grok_api_key, model_name = model_judge)
       fact_checker_chain = (
           {
               "question": RunnablePassthrough(),
               "evidence": context_retriever 
           }
           | fact_checker_prompt
           | llm_fact_checker
           | StrOutputParser()
       )
   
       claim = claim1
       
       #Calling SARVAM API to translate Indic languages to English
       client = SarvamAI(api_subscription_key = sarvam_api_key)
       
       try:
           translation = client.text.translate(
           input=claim,
           source_language_code="auto",
           target_language_code="en-IN"
           )
       except Exception as e:
           print(f"Error during translation: {e}")
           error_msg = 'It appears you have provided input in an alien language. Please try again with some other language'
           return error_msg,error_msg,error_msg
       
       claim_final = translation.translated_text if translation else claim
       claim_orig_lang = translation.source_language_code
       #print(f"Translated claim: {claim_final}")
       
       verdict_orig = fact_checker_chain.invoke(claim_final)
       verdict_class = verdict_orig.split('\n')[0]
       verdict_explan = verdict_orig.split('\n')[-1]

       # Strip out any <think> or <tool> or similar tags
       cleaned_output = re.sub(r"<[^>]+>", "", verdict_orig).strip()
       
       verdict_class = re.search(r'Classification:\s*(REAL|FAKE|UNSURE)', cleaned_output)
       verdict_explan = re.search(r'Explanation:\s*(.*)', cleaned_output, re.DOTALL)
       
       verdict_class = verdict_class.group(1) if verdict_class else "UNSURE"
       verdict_explan = verdict_explan.group(1).strip() if verdict_explan else "No clear explanation found."

       #print('here1: ',verdict_class)
       #print('here2: ',verdict_explan)
       
       if input_lang == 'auto':
           trans_lang = claim_orig_lang
       else:
           trans_lang = input_lang
   
       if claim_orig_lang != 'en-IN':
           try:
               translation_class = client.text.translate(
               input=verdict_class,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
               
           try:
               translation_explan = client.text.translate(
               input=verdict_explan,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
           
           verdict_trans_class = translation_class.translated_text
           verdict_trans_explan = translation_explan.translated_text
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           
           verdict_orig = verdict_class + '\n\n' + verdict_explan
       else:
           verdict_orig = verdict_class + '\n\n' + verdict_explan
           verdict_trans_class = verdict_class
           verdict_trans_explan = verdict_explan
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           #print('verdict_orig: ', verdict_orig) 
           
           try:
               match = re.search(r':(.*)', verdict_class)
               if match:
                    verdict1 = match.group(1).strip()
                    #print(verdict1)
           except Exception as e:
               print('here3')
               verdict1 = verdict_class
       
       #return(claim_final, verdict_orig, verdict_trans)
       return(claim_final, verdict_class, verdict_orig)
    except Exception as e:
        if str(e) == 'No result found':
            print('Error in main proc1. Error is ', e)
            error_msg = 'The search for this claim came back empty. Please rephrase the claim or try with a new one'
            return('UNSURE' ,error_msg,error_msg)
        else:
            print('Error in main proc2. Error is ', e)
            error_msg = 'Something went wrong. Please try after some time'
            return(error_msg,error_msg,error_msg)
    
def transcribe_audio(audio):
    """
       Description: This function trascibes audio using SarvamAI STT model 
    """
    try:
        client = SarvamAI(api_subscription_key = sarvam_api_key)
        mime_type, _ = mimetypes.guess_type(audio)
        
        with open(audio, "rb") as f:
            response = client.speech_to_text.transcribe(
                file=("audio.mp3", f, mime_type or "audio/mpeg"),
                model="saarika:v2.5",
                language_code="unknown"
            )
        ret_var = response.transcript
        ret_lang = response.language_code
    except Exception as e:
        print(f"Error during translation: {e}")
        ret_var = ''

    return ret_var, ret_lang
    
def verify_news_audio(audio):
    """
       Description: This function verifies the news where input method is Audio
    """
    
    claim, orig_lang = transcribe_audio(audio)
    if claim == '':
        error_msg = 'I could not understand your message. Please try recording again'
        return(error_msg,error_msg,error_msg)
    
    final_claim, verdict, verdict_trans = verify_news(claim, orig_lang)
    return final_claim, verdict, verdict_trans

In [ ]:
df_strategy_2 = model_evaluate()
df_strategy_2

Evaluating with Prompt Strategy 3
    
    Vanilla RAG  
    Summarization  
    Judge Prompt  

In [14]:
import requests
import mimetypes
import os
import json

from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

grok_api_key = os.getenv('GROK_API_KEY')
serp_dev_api_key = os.getenv('SERP_DEV_API_KEY')
sarvam_api_key = os.getenv('SARVAM_API_KEY')

empty_search = None

class SerperRetrieverWrapper_3:
    #Class to use Serper as retriver agent for the RAG framework
    def __init__(self, api_key: str, num_results: int = 15):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        """
        try:
            _SERPER_SEARCH_URL = "https://google.serper.dev/news"
            q1 = '(site:news18.com OR site:ptinews.com OR site:politifact.com) ' + query
            #print('Serper query: ', q1)
    
            headers = {
                "X-API-KEY": self.api_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": q1,
                "num": self.num_results,
            }
            
            resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload)
            #print('Serper response: ', resp)
            #print('Serper response code: ', resp.json())
            
            if resp.status_code != 200:
                raise Exception(f"Serper API Error: {resp.text}")
            results = resp.json()
            #print('Serper results: ', results.type)

            #return results
            
            documents = []
            for i, item in enumerate(results.get("news", []), 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')
                documents.append(Document(
                    page_content=f"{title}\n{snippet}",
                    metadata={"source": link}
                ))
            
            #if not documents:
            #    empty_search = Exception("No result found")
            #    raise empty_search
            
            #print(f"Retrieved {len(documents)} documents from Serper.")
            return documents
        except Exception as e:
            if e is empty_search:
                print('serper main exception1')
                raise empty_search
            else:
                print('serper main exception2')
                raise e
        
def verify_news(user_claim, i_model_multi_query, i_model_summarizer, i_model_judge, input_lang = 'auto'):
    """
       Description: This function is used to verify the claim provided by the user and output as REAL or FAKE or UNSURE based on the context with a short explanation
       INPUT: user_claim --> The news user wish to verify
       OUTPUT: FAKE/REAL/UNSURE with explanation
    """
    try:
       model_multi_query = i_model_multi_query
       model_summarizer = i_model_summarizer
       model_judge = i_model_judge

       #print('here1: ', user_claim)
       claim1 = user_claim.replace("'","")
       claim1 = user_claim.replace("\n"," ")
       #print('here2: ', claim1)
        
       serper_retriever = SerperRetrieverWrapper_3(api_key=serp_dev_api_key)
       context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)
       
       """
       Implemeting RAG framework with Multi Query Translation
       Step 1 -- Take input from user for the news to verify
       Step 2 -- Generate 3 variants of the news for better seach results
       Step 3 -- Summarise the results from all the previous steps to be passed to main prompt
       Step 4 -- Deliver the final verdict with a short explanation
       """
       
       '''
       #Multi Query Generation
       
       multi_query_template = """
            You are an AI language assistant.
       
             Your task is to generate three alternative phrasings of the given user question to help retrieve relevant documents from a vector database. These alternatives should reflect different ways the original question might be asked, using varied vocabulary or structure, while preserving the original intent.             
             This helps improve the chances of matching relevant content in a distance-based similarity search.
             
             Original question: {question}
             
             Output:
             Provide exactly three reworded versions of the question, each on a new line.
       """
       perspectives_prompt = ChatPromptTemplate.from_template(multi_query_template)
       
       llm_multi_query = ChatGroq(api_key = grok_api_key, model_name = model_multi_query)
       
       generate_queries = (
           perspectives_prompt 
           | llm_multi_query
           | StrOutputParser() 
           | (lambda x: x.split("\n"))
       )
       '''
       #Summarization using multi query
       
       summarizer_template = '''
          You are an assistant summarizing factual evidence from multiple documents.
       
          Based on the following documents, extract the key facts relevant to the claim.
          
          Claim: {question}
          
          Documents:
          {context}
          
          Return a short neutral summary of the key facts only.
       '''
       summarizer_prompt = PromptTemplate.from_template(summarizer_template)
       
       llm_summarizer = ChatGroq(api_key = grok_api_key, model_name = model_summarizer)
       summarizer_chain = (
           {
               "context": context_retriever,
               "question": RunnablePassthrough()
           }
           | summarizer_prompt
           | llm_summarizer
           | StrOutputParser()
       )
       
       #Final Judgement 
       
       fact_checker_template = '''
          You are a fact-checking assistant. Give a direct answer without showing your thinking process.
          
          Claim: {question}
          
          Evidence:
          {evidence}
          
          Instructions:
          
          If evidence is provided, use only that evidence to determine whether the claim is REAL, FAKE, or UNSURE.          
          If no evidence is provided, conduct a brief web search to gather supporting or contradicting information. Prioritize information from reliable, reputable sources such as news websites, fact-checking organizations, or official statements.
          
          If no sufficient or conclusive evidence is found from these sources, respond with UNSURE.          
          Do not speculate or rely on untrusted sources.

          Respond in this format:
          
          Classification: REAL / FAKE / UNSURE  
          Explanation: <Your reasoning, clearly supported by the evidence or source content> 
       '''
       
       fact_checker_prompt = PromptTemplate.from_template(fact_checker_template)
       
       llm_fact_checker = ChatGroq(api_key = grok_api_key, model_name = model_judge)
       fact_checker_chain = (
           {
               "question": RunnablePassthrough(),
               "evidence": summarizer_chain 
           }
           | fact_checker_prompt
           | llm_fact_checker
           | StrOutputParser()
       )
   
       claim = claim1
       
       #Calling SARVAM API to translate Indic languages to English
       client = SarvamAI(api_subscription_key = sarvam_api_key)
       
       try:
           translation = client.text.translate(
           input=claim,
           source_language_code="auto",
           target_language_code="en-IN"
           )
       except Exception as e:
           print(f"Error during translation: {e}")
           error_msg = 'It appears you have provided input in an alien language. Please try again with some other language'
           return error_msg,error_msg,error_msg
       
       claim_final = translation.translated_text if translation else claim
       claim_orig_lang = translation.source_language_code
       #print(f"Translated claim: {claim_final}")
       
       verdict_orig = fact_checker_chain.invoke(claim_final)
       verdict_class = verdict_orig.split('\n')[0]
       verdict_explan = verdict_orig.split('\n')[-1]

       # Strip out any <think> or <tool> or similar tags
       cleaned_output = re.sub(r"<[^>]+>", "", verdict_orig).strip()
       
       verdict_class = re.search(r'Classification:\s*(REAL|FAKE|UNSURE)', cleaned_output)
       verdict_explan = re.search(r'Explanation:\s*(.*)', cleaned_output, re.DOTALL)
       
       verdict_class = verdict_class.group(1) if verdict_class else "UNSURE"
       verdict_explan = verdict_explan.group(1).strip() if verdict_explan else "No clear explanation found."

       #print('here1: ',verdict_class)
       #print('here2: ',verdict_explan)
       
       if input_lang == 'auto':
           trans_lang = claim_orig_lang
       else:
           trans_lang = input_lang
   
       if claim_orig_lang != 'en-IN':
           try:
               translation_class = client.text.translate(
               input=verdict_class,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
               
           try:
               translation_explan = client.text.translate(
               input=verdict_explan,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
           
           verdict_trans_class = translation_class.translated_text
           verdict_trans_explan = translation_explan.translated_text
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           
           verdict_orig = verdict_class + '\n\n' + verdict_explan
       else:
           verdict_orig = verdict_class + '\n\n' + verdict_explan
           verdict_trans_class = verdict_class
           verdict_trans_explan = verdict_explan
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           #print('verdict_orig: ', verdict_orig) 
           
           try:
               match = re.search(r':(.*)', verdict_class)
               if match:
                    verdict1 = match.group(1).strip()
                    #print(verdict1)
           except Exception as e:
               print('here3')
               verdict1 = verdict_class
       
       #return(claim_final, verdict_orig, verdict_trans)
       return(claim_final, verdict_class, verdict_orig)
    except Exception as e:
        if str(e) == 'No result found':
            print('Error in main proc1. Error is ', e)
            error_msg = 'The search for this claim came back empty. Please rephrase the claim or try with a new one'
            return('UNSURE' ,error_msg,error_msg)
        else:
            print('Error in main proc2. Error is ', e)
            error_msg = 'Something went wrong. Please try after some time'
            return(error_msg,error_msg,error_msg)
    
def transcribe_audio(audio):
    """
       Description: This function trascibes audio using SarvamAI STT model 
    """
    try:
        client = SarvamAI(api_subscription_key = sarvam_api_key)
        mime_type, _ = mimetypes.guess_type(audio)
        
        with open(audio, "rb") as f:
            response = client.speech_to_text.transcribe(
                file=("audio.mp3", f, mime_type or "audio/mpeg"),
                model="saarika:v2.5",
                language_code="unknown"
            )
        ret_var = response.transcript
        ret_lang = response.language_code
    except Exception as e:
        print(f"Error during translation: {e}")
        ret_var = ''

    return ret_var, ret_lang
    
def verify_news_audio(audio):
    """
       Description: This function verifies the news where input method is Audio
    """
    
    claim, orig_lang = transcribe_audio(audio)
    if claim == '':
        error_msg = 'I could not understand your message. Please try recording again'
        return(error_msg,error_msg,error_msg)
    
    final_claim, verdict, verdict_trans = verify_news(claim, orig_lang)
    return final_claim, verdict, verdict_trans

In [ ]:
df_strategy_3 = model_evaluate()
df_strategy_3

Evaulating with Regional languages using Prompt Strategy 1

In [20]:
import requests
import mimetypes
import os
import json

from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

empty_search = None

class SerperRetrieverWrapper:
    #Class to use Serper as retriver agent for the RAG framework
    def __init__(self, api_key: str, num_results: int = 15):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        """
        try:
            _SERPER_SEARCH_URL = "https://google.serper.dev/news"
            q1 = '(site:news18.com OR site:ptinews.com OR site:politifact.com) ' + query
            #print('Serper query: ', q1)
    
            headers = {
                "X-API-KEY": self.api_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": q1,
                "num": self.num_results,
            }
            
            resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload)
            #print('Serper response: ', resp)
            #print('Serper response code: ', resp.json())
            
            if resp.status_code != 200:
                raise Exception(f"Serper API Error: {resp.text}")
            results = resp.json()
            #print('Serper results: ', results.type)

            #return results
            
            documents = []
            for i, item in enumerate(results.get("news", []), 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')
                documents.append(Document(
                    page_content=f"{title}\n{snippet}",
                    metadata={"source": link}
                ))
            
            #if not documents:
            #    empty_search = Exception("No result found")
            #    raise empty_search
            
            #print(f"Retrieved {len(documents)} documents from Serper.")
            return documents
        except Exception as e:
            if e is empty_search:
                print('serper main exception1')
                raise empty_search
            else:
                print('serper main exception2')
                raise e
        
def verify_news(user_claim, i_model_multi_query, i_model_summarizer, i_model_judge, input_lang = 'auto'):
    """
       Description: This function is used to verify the claim provided by the user and output as REAL or FAKE or UNSURE based on the context with a short explanation
       INPUT: user_claim --> The news user wish to verify
       OUTPUT: FAKE/REAL/UNSURE with explanation
    """
    try:
       model_multi_query = i_model_multi_query
       model_summarizer = i_model_summarizer
       model_judge = i_model_judge

       #print('here1: ', user_claim)
       claim1 = user_claim.replace("'","")
       claim1 = claim1.replace("\n"," ")
       #print('here2: ', claim1)
        
       serper_retriever = SerperRetrieverWrapper(api_key=serp_dev_api_key)
       context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)
       
       """
       Implemeting RAG framework with Multi Query Translation
       Step 1 -- Take input from user for the news to verify
       Step 2 -- Generate 3 variants of the news for better seach results
       Step 3 -- Summarise the results from all the previous steps to be passed to main prompt
       Step 4 -- Deliver the final verdict with a short explanation
       """
       
       #Multi Query Generation
       
       multi_query_template = """
            You are an AI language assistant.

             Your task is to generate three alternative phrasings of the given user question to help retrieve relevant documents from a vector database. These alternatives should reflect different ways the original question might be asked, using varied vocabulary or structure, while preserving the original intent.             
             This helps improve the chances of matching relevant content in a distance-based similarity search.
             
             Original question: {question}
             
             Output:
             Provide exactly three reworded versions of the question, each on a new line.
       """
       perspectives_prompt = ChatPromptTemplate.from_template(multi_query_template)
       
       llm_multi_query = ChatGroq(api_key = grok_api_key, model_name = model_multi_query)
       
       generate_queries = (
           perspectives_prompt 
           | llm_multi_query
           | StrOutputParser() 
           | (lambda x: x.split("\n"))
       )
       
       """
       #Summarization using multi query
       
       summarizer_template = '''
          You are an assistant summarizing factual evidence from multiple documents.
       
          Based on the following documents, extract the key facts relevant to the claim.
          
          Claim: {question}
          
          Documents:
          {context}
          
          Return a short neutral summary of the key facts only.
       '''
       summarizer_prompt = PromptTemplate.from_template(summarizer_template)
       
       llm_summarizer = ChatGroq(api_key = grok_api_key, model_name = model_summarizer)
       summarizer_chain = (
           {
               "context": context_retriever,
               "question": generate_queries
           }
           | summarizer_prompt
           | llm_summarizer
           | StrOutputParser()
       )
       """
       
       #Final Judgement 
       
       fact_checker_template = '''
          You are a fact-checking assistant.
          
          Claim: {question}
          
          Evidence:
          {evidence}
          
          Instructions:
          
          If evidence is provided, use only that evidence to determine whether the claim is REAL, FAKE, or UNSURE.          
          
          If no sufficient or conclusive evidence is found from these sources, respond with UNSURE.          
          Do not speculate or rely on untrusted sources.

          Respond in this format:
          
          Classification: REAL / FAKE / UNSURE  
          Explanation: <Your reasoning, clearly supported by the evidence or source content> 
       '''
       
       fact_checker_prompt = PromptTemplate.from_template(fact_checker_template)
       
       llm_fact_checker = ChatGroq(api_key = grok_api_key, model_name = model_judge)
       fact_checker_chain = (
           {
               "question": RunnablePassthrough(),
               "evidence": context_retriever 
           }
           | fact_checker_prompt
           | llm_fact_checker
           | StrOutputParser()
       )
   
       claim = claim1
       
       #Calling SARVAM API to translate Indic languages to English
       client = SarvamAI(api_subscription_key = sarvam_api_key)
       
       try:
           translation = client.text.translate(
           input=claim,
           source_language_code="auto",
           target_language_code="en-IN"
           )
       except Exception as e:
           print(f"Error during translation: {e}")
           error_msg = 'It appears you have provided input in an alien language. Please try again with some other language'
           return error_msg,error_msg,error_msg
       
       claim_final = translation.translated_text if translation else claim
       claim_orig_lang = translation.source_language_code
       #print(f"Translated claim: {claim_final}")
       
       verdict_orig = fact_checker_chain.invoke(claim_final)
       verdict_class = verdict_orig.split('\n')[0]
       verdict_explan = verdict_orig.split('\n')[-1]

       # Strip out any <think> or <tool> or similar tags
       cleaned_output = re.sub(r"<[^>]+>", "", verdict_orig).strip()
       
       verdict_class = re.search(r'Classification:\s*(REAL|FAKE|UNSURE)', cleaned_output)
       verdict_explan = re.search(r'Explanation:\s*(.*)', cleaned_output, re.DOTALL)
       
       verdict_class = verdict_class.group(1) if verdict_class else "UNSURE"
       verdict_explan = verdict_explan.group(1).strip() if verdict_explan else "No clear explanation found."

       #print('here1: ',verdict_class)
       #print('here2: ',verdict_explan)
       
       if input_lang == 'auto':
           trans_lang = claim_orig_lang
       else:
           trans_lang = input_lang
   
       if claim_orig_lang != 'en-IN':
           try:
               translation_class = client.text.translate(
               input=verdict_class,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
               
           try:
               translation_explan = client.text.translate(
               input=verdict_explan,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
           
           verdict_trans_class = translation_class.translated_text
           verdict_trans_explan = translation_explan.translated_text
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           
           verdict_orig = verdict_class + '\n\n' + verdict_explan
       else:
           verdict_orig = verdict_class + '\n\n' + verdict_explan
           verdict_trans_class = verdict_class
           verdict_trans_explan = verdict_explan
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           #print('verdict_orig: ', verdict_orig) 
           
           try:
               match = re.search(r':(.*)', verdict_class)
               if match:
                    verdict1 = match.group(1).strip()
                    #print(verdict1)
           except Exception as e:
               print('here3')
               verdict1 = verdict_class
       
       #return(claim_final, verdict_orig, verdict_trans)
       return(claim_final, verdict_class, verdict_orig)
    except Exception as e:
        if str(e) == 'No result found':
            print('Error in main proc1. Error is ', e)
            error_msg = 'The search for this claim came back empty. Please rephrase the claim or try with a new one'
            return('UNSURE' ,error_msg,error_msg)
        else:
            print('Error in main proc2. Error is ', e)
            error_msg = 'Something went wrong. Please try after some time'
            return(error_msg,error_msg,error_msg)
    
def transcribe_audio(audio):
    """
       Description: This function trascibes audio using SarvamAI STT model 
    """
    try:
        client = SarvamAI(api_subscription_key = sarvam_api_key)
        mime_type, _ = mimetypes.guess_type(audio)
        
        with open(audio, "rb") as f:
            response = client.speech_to_text.transcribe(
                file=("audio.mp3", f, mime_type or "audio/mpeg"),
                model="saarika:v2.5",
                language_code="unknown"
            )
        ret_var = response.transcript
        ret_lang = response.language_code
    except Exception as e:
        print(f"Error during translation: {e}")
        ret_var = ''

    return ret_var, ret_lang
    
def verify_news_audio(audio):
    """
       Description: This function verifies the news where input method is Audio
    """
    
    claim, orig_lang = transcribe_audio(audio)
    if claim == '':
        error_msg = 'I could not understand your message. Please try recording again'
        return(error_msg,error_msg,error_msg)
    
    final_claim, verdict, verdict_trans = verify_news(claim, orig_lang)
    return final_claim, verdict, verdict_trans

In [ ]:
df_test_regional = model_evaluate('fake_news_claims_factchecked_reginal.csv')
df_test_regional

Evaluating performance with input as Audio

In [22]:
import requests
import mimetypes
import os
import json

from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

grok_api_key = os.getenv('GROK_API_KEY')
serp_dev_api_key = os.getenv('SERP_DEV_API_KEY')
sarvam_api_key = os.getenv('SARVAM_API_KEY')

empty_search = None

class SerperRetrieverWrapper:
    #Class to use Serper as retriver agent for the RAG framework
    def __init__(self, api_key: str, num_results: int = 15):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        """
        try:
            _SERPER_SEARCH_URL = "https://google.serper.dev/news"
            q1 = '(site:news18.com OR site:ptinews.com OR site:politifact.com) ' + query
            #print('Serper query: ', q1)
    
            headers = {
                "X-API-KEY": self.api_key,
                "Content-Type": "application/json"
            }
            payload = {
                "q": q1,
                "num": self.num_results,
            }
            
            resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload)
            #print('Serper response: ', resp)
            #print('Serper response code: ', resp.json())
            
            if resp.status_code != 200:
                raise Exception(f"Serper API Error: {resp.text}")
            results = resp.json()
            #print('Serper results: ', results.type)

            #return results
            
            documents = []
            for i, item in enumerate(results.get("news", []), 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')
                documents.append(Document(
                    page_content=f"{title}\n{snippet}",
                    metadata={"source": link}
                ))
            
            #if not documents:
            #    empty_search = Exception("No result found")
            #    raise empty_search
            
            #print(f"Retrieved {len(documents)} documents from Serper.")
            return documents
        except Exception as e:
            if e is empty_search:
                print('serper main exception1')
                raise empty_search
            else:
                print('serper main exception2')
                raise e
        
def verify_news(user_claim, i_model_multi_query, i_model_summarizer, i_model_judge, input_lang = 'auto'):
    """
       Description: This function is used to verify the claim provided by the user and output as REAL or FAKE or UNSURE based on the context with a short explanation
       INPUT: user_claim --> The news user wish to verify
       OUTPUT: FAKE/REAL/UNSURE with explanation
    """
    try:
       model_multi_query = i_model_multi_query
       model_summarizer = i_model_summarizer
       model_judge = i_model_judge

       #print('here1: ', user_claim)
       claim1 = user_claim.replace("'","")
       claim1 = claim1.replace("\n"," ")
       #print('here2: ', claim1)
        
       serper_retriever = SerperRetrieverWrapper(api_key=serp_dev_api_key)
       context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)
       
       """
       Implemeting RAG framework with Multi Query Translation
       Step 1 -- Take input from user for the news to verify
       Step 2 -- Generate 3 variants of the news for better seach results
       Step 3 -- Summarise the results from all the previous steps to be passed to main prompt
       Step 4 -- Deliver the final verdict with a short explanation
       """
       
       #Multi Query Generation
       
       multi_query_template = """
            You are an AI language assistant.

             Your task is to generate three alternative phrasings of the given user question to help retrieve relevant documents from a vector database. These alternatives should reflect different ways the original question might be asked, using varied vocabulary or structure, while preserving the original intent.             
             This helps improve the chances of matching relevant content in a distance-based similarity search.
             
             Original question: {question}
             
             Output:
             Provide exactly three reworded versions of the question, each on a new line.
       """
       perspectives_prompt = ChatPromptTemplate.from_template(multi_query_template)
       
       llm_multi_query = ChatGroq(api_key = grok_api_key, model_name = model_multi_query)
       
       generate_queries = (
           perspectives_prompt 
           | llm_multi_query
           | StrOutputParser() 
           | (lambda x: x.split("\n"))
       )
       
       """
       #Summarization using multi query
       
       summarizer_template = '''
          You are an assistant summarizing factual evidence from multiple documents.
       
          Based on the following documents, extract the key facts relevant to the claim.
          
          Claim: {question}
          
          Documents:
          {context}
          
          Return a short neutral summary of the key facts only.
       '''
       summarizer_prompt = PromptTemplate.from_template(summarizer_template)
       
       llm_summarizer = ChatGroq(api_key = grok_api_key, model_name = model_summarizer)
       summarizer_chain = (
           {
               "context": context_retriever,
               "question": generate_queries
           }
           | summarizer_prompt
           | llm_summarizer
           | StrOutputParser()
       )
       """
       
       #Final Judgement 
       
       fact_checker_template = '''
          You are a fact-checking assistant.
          
          Claim: {question}
          
          Evidence:
          {evidence}
          
          Instructions:
          
          If evidence is provided, use only that evidence to determine whether the claim is REAL, FAKE, or UNSURE.          
          
          If no sufficient or conclusive evidence is found from these sources, respond with UNSURE.          
          Do not speculate or rely on untrusted sources.

          Respond in this format:
          
          Classification: REAL / FAKE / UNSURE  
          Explanation: <Your reasoning, clearly supported by the evidence or source content> 
       '''
       
       fact_checker_prompt = PromptTemplate.from_template(fact_checker_template)
       
       llm_fact_checker = ChatGroq(api_key = grok_api_key, model_name = model_judge)
       fact_checker_chain = (
           {
               "question": RunnablePassthrough(),
               "evidence": context_retriever 
           }
           | fact_checker_prompt
           | llm_fact_checker
           | StrOutputParser()
       )
   
       claim = claim1
       
       #Calling SARVAM API to translate Indic languages to English
       client = SarvamAI(api_subscription_key = sarvam_api_key)
       
       try:
           translation = client.text.translate(
           input=claim,
           source_language_code="auto",
           target_language_code="en-IN"
           )
       except Exception as e:
           print(f"Error during translation: {e}")
           error_msg = 'It appears you have provided input in an alien language. Please try again with some other language'
           return error_msg,error_msg,error_msg
       
       claim_final = translation.translated_text if translation else claim
       claim_orig_lang = translation.source_language_code
       #print(f"Translated claim: {claim_final}")
       
       verdict_orig = fact_checker_chain.invoke(claim_final)
       verdict_class = verdict_orig.split('\n')[0]
       verdict_explan = verdict_orig.split('\n')[-1]

       # Strip out any <think> or <tool> or similar tags
       cleaned_output = re.sub(r"<[^>]+>", "", verdict_orig).strip()
       
       verdict_class = re.search(r'Classification:\s*(REAL|FAKE|UNSURE)', cleaned_output)
       verdict_explan = re.search(r'Explanation:\s*(.*)', cleaned_output, re.DOTALL)
       
       verdict_class = verdict_class.group(1) if verdict_class else "UNSURE"
       verdict_explan = verdict_explan.group(1).strip() if verdict_explan else "No clear explanation found."

       #print('here1: ',verdict_class)
       #print('here2: ',verdict_explan)
       
       if input_lang == 'auto':
           trans_lang = claim_orig_lang
       else:
           trans_lang = input_lang
   
       if claim_orig_lang != 'en-IN':
           try:
               translation_class = client.text.translate(
               input=verdict_class,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
               
           try:
               translation_explan = client.text.translate(
               input=verdict_explan,
               source_language_code='en-IN',
               target_language_code=claim_orig_lang
               )
           except Exception as e:
               print(f"Error during verdict translation: {e}")  
               error_msg = 'Something went wrong while translating the verdict. Please try again'
               return error_msg,error_msg,error_msg
           
           verdict_trans_class = translation_class.translated_text
           verdict_trans_explan = translation_explan.translated_text
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           
           verdict_orig = verdict_class + '\n\n' + verdict_explan
       else:
           verdict_orig = verdict_class + '\n\n' + verdict_explan
           verdict_trans_class = verdict_class
           verdict_trans_explan = verdict_explan
           verdict_trans = verdict_trans_class + '\n\n' + verdict_trans_explan
           #print('verdict_orig: ', verdict_orig) 
           
           try:
               match = re.search(r':(.*)', verdict_class)
               if match:
                    verdict1 = match.group(1).strip()
                    #print(verdict1)
           except Exception as e:
               print('here3')
               verdict1 = verdict_class
       
       #return(claim_final, verdict_orig, verdict_trans)
       return(claim_final, verdict_class, verdict_orig)
    except Exception as e:
        if str(e) == 'No result found':
            print('Error in main proc1. Error is ', e)
            error_msg = 'The search for this claim came back empty. Please rephrase the claim or try with a new one'
            return('UNSURE' ,error_msg,error_msg)
        else:
            print('Error in main proc2. Error is ', e)
            error_msg = 'Something went wrong. Please try after some time'
            return(error_msg,error_msg,error_msg)
    
def transcribe_audio(audio):
    """
       Description: This function trascibes audio using SarvamAI STT model 
    """
    try:
        client = SarvamAI(api_subscription_key = sarvam_api_key)
        mime_type, _ = mimetypes.guess_type(audio)
        
        with open(audio, "rb") as f:
            response = client.speech_to_text.transcribe(
                file=("audio.mp3", f, mime_type or "audio/mpeg"),
                model="saarika:v2.5",
                language_code="unknown"
            )
        ret_var = response.transcript
        ret_lang = response.language_code
    except Exception as e:
        print(f"Error during translation: {e}")
        ret_var = ''

    return ret_var, ret_lang
    
def verify_news_audio(audio,i_model_multi_query, i_model_summarizer, i_model_judge):
    """
       Description: This function verifies the news where input method is Audio
    """
    
    claim, orig_lang = transcribe_audio(audio)
    if claim == '':
        error_msg = 'I could not understand your message. Please try recording again'
        return(error_msg,error_msg,error_msg)
    
    final_claim, verdict, verdict_trans = verify_news(claim, i_model_multi_query, i_model_summarizer, i_model_judge,orig_lang)
    return final_claim, verdict, verdict_trans

In [ ]:
df_res = pd.DataFrame(columns=['model_judge','claim_trans','veridict_class','veridict_full','file_name'])
for i in os.listdir('C:\\Users\\rahul\\OneDrive\\7_Learning\\IISC\\Courses\\3.1_Deep_Learning\\Course Material\\Project\\wip\\audio\\'):
    if i.endswith('.mp3'):
        print('Processing file: ', i)
        audio_path = os.path.join('C:\\Users\\rahul\\OneDrive\\7_Learning\\IISC\\Courses\\3.1_Deep_Learning\\Course Material\\Project\\wip\\audio', i)
        #models = ['qwen-qwq-32b','llama3-8b-8192','qwen/qwen3-32b','mistral-saba-24b','deepseek-r1-distill-llama-70b',
        #  'meta-llama/llama-4-scout-17b-16e-instruct',  'meta-llama/llama-4-maverick-17b-128e-instruct']

        models = ['qwen-qwq-32b','llama3-8b-8192','mistral-saba-24b']
        #  'meta-llama/llama-4-scout-17b-16e-instruct',  'meta-llama/llama-4-maverick-17b-128e-instruct']

        df_model_res =pd.DataFrame(columns=['model_multi_query', 'model_summarizer', 'model_judge', 'abstention_rate', 'accuracy', 'overall_accuracy', 'coverage', 'abstention_rate', 'f1_real', 'f1_fake'])    
        
        
        for judge_model in models:
             model_multi_query = models[0]  # Keep constant
             model_summarizer = models[1]   # Keep constant  
             model_judge = judge_model      # Test each one
             print(f"Testing with judge: {judge_model}")
            
             claim_trans, veridict_class, verdict_full = verify_news_audio(audio_path, model_multi_query, model_summarizer, model_judge)

             lis1 = [model_judge , claim_trans, veridict_class, verdict_full, i]
             df_res.loc[len(df_res)] = lis1
        
            

In [24]:
df_res

,model_judge,claim_trans,veridict_class,veridict_full,file_name
0,qwen-qwq-32b,The 2000 rupee note has a GPS tracker.,UNSURE,"UNSURE\n\nThe provided evidence consists of Politifact fact-checks focused on U.S. political topics such as immigration, voter fraud, job creation, and policy history. None of the documents mention India's 2000 rupee note or GPS trackers in currency. Since the evidence does not address the claim, there is no basis to confirm or refute it using the given sources.",2000.mp3


In [25]:
##For all distnct model judge run below codes manually